In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
import pandas as pd
import numpy as np
import openai
from datetime import datetime
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import PGVector
from dotenv import load_dotenv
# from langchain_community.retrievers import MergerRetriever  

# 변경 부분 - Google genai, and PGVector
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_postgres.vectorstores import PGVector
from langchain_community.document_loaders import DirectoryLoader, TextLoader
import time
from dotenv import load_dotenv


#Openai 추가
from langchain_openai import OpenAIEmbeddings
from langchain_postgres import PGVector
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

from langchain_core.documents import Document
from tqdm import tqdm
import time

import re
import json
from typing import Dict, List
from datetime import datetime

from IPython.display import display, HTML
import re
import requests
import base64


In [3]:
# API KEY 체크
load_dotenv(override=True)

# API 키 확인
api_key = os.getenv('OPENAI_API_KEY')
print(f"API 키: {api_key[:8]}")
CONNECTION_STRING = os.getenv("DATABASE_URL")
print(f"DB connection string : {CONNECTION_STRING[:3]}")

# 만약 None이면 문제!
if api_key is None:
    print("❌ .env 파일에 OPENAI_API_KEY가 없습니다!")

if CONNECTION_STRING is None:
    print("❌ .env 파일에 CONNECTION_STRING가 없습니다!")
 

API 키: sk-proj-
DB connection string : pos


In [4]:


# 설정

COLLECTION_NAME = 'bookstack_v2'

class ParentChildRAGChain:
    """부모-자식 구조 RAG 체인 (기본)"""
    
    def __init__(self, connection_string: str, collection_name: str):
        self.embeddings = OpenAIEmbeddings()
        self.llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
        
        self.vectorstore = PGVector(
            embeddings=self.embeddings,
            collection_name=collection_name,
            connection=connection_string,
            use_jsonb=True
        )
        
        print(f"✅ RAG 체인 준비 완료 (컬렉션: {collection_name})\n")
    
    def search_top_parents(self, query: str) -> Dict:
        
        print(f"🔍 검색 중: '{query}'")
        
        # 자식 문서 검색
        child_results = self.vectorstore.similarity_search_with_score(
            query, k=10, filter={'doc_type': 'child'}
        )
        
        # 부모 문서도 같이 검색 (자식 없는 섹션 커버)
        parent_results = self.vectorstore.similarity_search_with_score(
            query, k=10, filter={'doc_type': 'parent'}
        )
        
        print(f"  → 자식 {len(child_results)}개 / 부모 {len(parent_results)}개 발견")
        
        # 자식+부모 합쳐서 score 기준 정렬
        all_results = child_results + parent_results
        all_results.sort(key=lambda x: x[1])  # score 낮을수록 유사도 높음
        
        if not all_results:
            return None
        
        best_doc, best_score = all_results[0]
        doc_type = best_doc.metadata.get('doc_type')
        
        print(f"  → 최고 점수: [{doc_type}] {best_doc.metadata.get('section', 'N/A')} (점수: {best_score:.4f})")
        
        # 상위 2개 부모 ID 수집
        seen_parent_ids = []
        for doc, score in all_results:
            if doc.metadata.get('doc_type') == 'child':
                parent_id = doc.metadata.get('parent_id')
            else:
                parent_id = doc.metadata.get('doc_id')
            
            if parent_id and parent_id not in seen_parent_ids:
                seen_parent_ids.append(parent_id)
            if len(seen_parent_ids) >= 2:
                break
        
        # 상위 2개 부모 문서 로드
        top2_parents = []
        for parent_id in seen_parent_ids:
            parent_docs = self.vectorstore.similarity_search(
                query, k=1, filter={'doc_id': parent_id}
            )
            if parent_docs:
                top2_parents.append(parent_docs[0])
        
        best_parent = top2_parents[0] if top2_parents else None
        best_child = best_doc if doc_type == 'child' else None
        
        if best_parent:
            print(f"  → 1위 부모: {best_parent.metadata.get('section', 'N/A')}")
        if len(top2_parents) > 1:
            print(f"  → 2위 부모: {top2_parents[1].metadata.get('section', 'N/A')}\n")
        
        return {
            'best_child': best_child,
            'best_parent': best_parent,
            'top2_parents': top2_parents,
            'score': best_score
        }
    
    def format_context(self, result: Dict) -> str:
        """1위 부모 문서만 LLM 컨텍스트로 포맷팅"""
        
        parent = result.get('best_parent')
        
        if parent:
            return f"""
[문서]
출처: {parent.metadata.get('book', 'N/A')}
섹션: {parent.metadata.get('section', 'N/A')}
유사도 점수: {result.get('score', 0):.4f}

{parent.page_content}
"""
        else:
            child = result.get('best_child')
            return f"""
[문서]
출처: {child.metadata.get('book', 'N/A')}
섹션: {child.metadata.get('section', 'N/A')}
유사도 점수: {result.get('score', 0):.4f}

{child.page_content}
"""


class ParentChildRAGWithRender(ParentChildRAGChain):
    """이미지 렌더링 포함 RAG (주피터용)"""
    
    def answer_with_render(self, query: str):
        """상위 2개 URL 먼저 표시 → 1위 부모로 LLM 답변 렌더링"""
        
        # 1. 검색
        result = self.search_top_parents(query)
        
        if not result:
            display(HTML("<p>관련 문서를 찾을 수 없습니다.</p>"))
            return
        
        # 2. 상위 2개 URL 카드 먼저 표시
        top2_parents = result.get('top2_parents', [])
        best_child = result.get('best_child')
        score = result.get('score', 0)
        
        # URL 카드 HTML 생성
        cards_html = '<div style="font-family: -apple-system, sans-serif; margin-bottom: 24px;">'
        cards_html += f'<p style="margin: 0 0 12px 0; font-size: 0.9em; color: #666;">📚 참고 문서 (상위 2개)</p>'
        
        docs_for_url = top2_parents if top2_parents else [best_child]
        
        for i, doc in enumerate(docs_for_url[:2]):
            source_url = doc.metadata.get('source_url', '')
            book_url = doc.metadata.get('book_url', '')
            section = doc.metadata.get('section', 'N/A')
            book = doc.metadata.get('book', 'N/A')
            url_display = source_url or book_url
            rank_label = "🥇 1위" if i == 0 else "🥈 2위"
            score_label = f"(유사도: {score:.4f})" if i == 0 else ""
            
            cards_html += f"""
            <div style="
                border: 1px solid {'#3498db' if i == 0 else '#bdc3c7'};
                border-radius: 8px;
                padding: 14px 18px;
                margin-bottom: 10px;
                background: {'#eaf4fd' if i == 0 else '#f8f9fa'};
            ">
                <p style="margin: 0 0 4px 0; font-size: 0.82em; color: #888;">{rank_label} {score_label}</p>
                <p style="margin: 0 0 6px 0; font-weight: bold; color: #2c3e50;">📖 {book} &gt; {section}</p>
                {"" if not url_display else f'<a href="{url_display}" target="_blank" style="color: #2980b9; font-size: 0.92em;">🔗 {url_display}</a>'}
            </div>
            """
        
        cards_html += '</div>'
        display(HTML(cards_html))
        
        # 3. 1위 부모만 LLM에 전달
        context = self.format_context(result)
        
        prompt = f"""You are an assistant for AS call center worker who needs information for error cases and technical support.
Use the following pieces of retrieved context to answer the question.

IMPORTANT RULES:
1. If you don't know the answer, just say that you don't know.
2. Provide several relevant cases with specific details.
3. Include step-by-step instructions when available.
4. ALWAYS include resource links (images, files) when they exist in the context.
5. End with one concise summary sentence.
6. Ignore documents without exact match - Priority is user's query. Strictly follow the query.
7. Answer in Korean.
8. do not delete any details including url or login information
9. if there is image information, explain them to be connected to text.

MOST important: 1. If you don't know the answer, just say that you don't know.

FORMAT YOUR ANSWER AS:
## 관련 사례

### 사례 1: [제목]
- 상황: [설명]
- 해결방법: [단계별 설명]
- 참고자료: [이미지/파일 링크가 있다면 반드시 포함]

### 사례 2: [제목]
...

## 요약
[한 문장 요약]

#Context: 
{context}

#Question:
{query}

#Answer:"""
        
        print(f"🤖 LLM 답변 생성 중...")
        messages = [{"role": "user", "content": prompt}]
        response = self.llm.invoke(messages)
        print(f"✅ 답변 완료\n")
        
        # 4. LLM 답변 렌더링
        html = self._convert_to_html(response.content)
        display(HTML(html))
    
    def _convert_to_html(self, md_text: str) -> str:
        """마크다운 → HTML (이미지 포함)"""
        html = md_text
        
        html = re.sub(r'^## (.+)$', r'<h2 style="color: #2c3e50; margin-top: 24px;">\1</h2>', html, flags=re.MULTILINE)
        html = re.sub(r'^### (.+)$', r'<h3 style="color: #34495e; margin-top: 20px;">\1</h3>', html, flags=re.MULTILINE)
        
        # 1. 정식 이미지 패턴 (![alt](url))
        image_pattern = r'!\[([^\]]*)\]\((http[s]?://[^\)]+\.(?:png|jpg|jpeg|gif|webp))\)'
        
        def img_replace(m):
            alt, url = m.groups()
            print(f"  📷 이미지: {alt or url[:50]}...", end=' ')
            base64_data = self._download_image_as_base64(url)
            if base64_data:
                print("✓")
                return f'''
                <div style="margin: 20px 0; padding: 15px; background: #f9f9f9; border-radius: 8px;">
                    <p style="margin: 0 0 10px 0; font-weight: bold; color: #555;">📷 {alt or "이미지"}</p>
                    <img src="{base64_data}" alt="{alt}" style="max-width: 100%; height: auto; border-radius: 4px; box-shadow: 0 2px 8px rgba(0,0,0,0.1);">
                </div>
                '''
            else:
                print("✗")
                return f'<p style="color: #999;">⚠️ 이미지: {alt or url}</p>'
        
        html = re.sub(image_pattern, img_replace, html)
        
        # 2. 링크 중 이미지 URL ([text](image.png))
        link_with_image_pattern = r'\[([^\]]+)\]\((http[s]?://[^\)]+\.(?:png|jpg|jpeg|gif|webp))\)'
        
        def link_img_replace(m):
            text, url = m.groups()
            print(f"  📷 링크형 이미지: {text}...", end=' ')
            base64_data = self._download_image_as_base64(url)
            if base64_data:
                print("✓")
                return f'''
                <div style="margin: 20px 0; padding: 15px; background: #f9f9f9; border-radius: 8px;">
                    <p style="margin: 0 0 10px 0; font-weight: bold; color: #555;">📷 {text}</p>
                    <img src="{base64_data}" alt="{text}" style="max-width: 100%; height: auto; border-radius: 4px; box-shadow: 0 2px 8px rgba(0,0,0,0.1);">
                </div>
                '''
            else:
                print("✗")
                return f'<a href="{url}" target="_blank" style="color: #2980b9;">{text}</a>'
        
        html = re.sub(link_with_image_pattern, link_img_replace, html)
        
        # 3. 나머지 일반 링크
        link_pattern = r'\[([^\]]+)\]\((http[s]?://[^\)]+)\)'
        html = re.sub(link_pattern, r'<a href="\2" target="_blank" style="color: #2980b9; border-bottom: 1px solid #2980b9;">\1</a>', html)
        
        # 리스트
        html = re.sub(r'^- (.+)$', r'<li>\1</li>', html, flags=re.MULTILINE)
        html = re.sub(r'^\d+\. (.+)$', r'<li>\1</li>', html, flags=re.MULTILINE)
        
        lines = html.split('\n')
        result = []
        in_list = False
        
        for line in lines:
            if line.strip().startswith('<li>'):
                if not in_list:
                    result.append('<ul style="margin: 12px 0; padding-left: 24px;">')
                    in_list = True
                result.append(line)
            else:
                if in_list:
                    result.append('</ul>')
                    in_list = False
                result.append(line)
        
        if in_list:
            result.append('</ul>')
        
        html = '\n'.join(result)
        html = html.replace('\n\n', '</p><p style="margin: 12px 0;">')
        html = f'<div style="font-family: -apple-system, sans-serif; line-height: 1.7; max-width: 900px;"><p style="margin: 12px 0;">{html}</p></div>'
        
        return html
    
    def _download_image_as_base64(self, url: str) -> str:
        """이미지 base64 변환"""
        try:
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                img_base64 = base64.b64encode(response.content).decode('utf-8')
                content_type = response.headers.get('Content-Type', 'image/png')
                return f"data:{content_type};base64,{img_base64}"
        except Exception:
            pass
        return None


# ============================================================
# 사용
# ============================================================

rag = ParentChildRAGWithRender(
    connection_string=CONNECTION_STRING,
    collection_name=COLLECTION_NAME
)

# rag.answer_with_render("락앤락 ENV 설정 메뉴 들어가는 방법")

✅ RAG 체인 준비 완료 (컬렉션: bookstack_v2)



In [5]:
# %%time
# rag.answer_with_render("정보보안 서약서")

In [6]:
# # 현재 노트북을 .py로 변환
# !jupyter nbconvert --to python Bookstack_family_Use_v3.ipynb --output ./python_scripts/Bookstack_family_Use_v3.py